# Scenarios generation

In [14]:

from xpress import *
import numpy as np
np.random.seed(8)  # Set the seed for reproducibility

def scenario_generation(N_scenarios):
    KT_values = {}  # Dictionary to store the generated values of KT
    for scenario in range(1, N_scenarios+1):
        KT_values[scenario] = np.random.poisson(100)  # Generate a Poisson-distributed value with mean 100
    return KT_values

# Accessing the KT values for a specific scenario
scenario_number = 1  # Example scenario number
KT_values = scenario_generation(100)
KT_value = KT_values[scenario_number]
#print(f"KT value for scenario {scenario_number}: {KT_value}")
print(KT_values)

{1: 113, 2: 92, 3: 98, 4: 101, 5: 102, 6: 108, 7: 103, 8: 95, 9: 87, 10: 82, 11: 97, 12: 96, 13: 120, 14: 98, 15: 110, 16: 81, 17: 109, 18: 99, 19: 90, 20: 91, 21: 101, 22: 121, 23: 103, 24: 100, 25: 98, 26: 90, 27: 103, 28: 94, 29: 111, 30: 106, 31: 111, 32: 113, 33: 94, 34: 100, 35: 110, 36: 120, 37: 89, 38: 102, 39: 110, 40: 87, 41: 112, 42: 92, 43: 84, 44: 99, 45: 91, 46: 99, 47: 101, 48: 111, 49: 103, 50: 89, 51: 104, 52: 114, 53: 96, 54: 77, 55: 102, 56: 101, 57: 99, 58: 109, 59: 89, 60: 92, 61: 101, 62: 101, 63: 119, 64: 93, 65: 103, 66: 92, 67: 104, 68: 103, 69: 89, 70: 106, 71: 104, 72: 96, 73: 94, 74: 96, 75: 94, 76: 90, 77: 90, 78: 97, 79: 94, 80: 113, 81: 91, 82: 93, 83: 102, 84: 110, 85: 118, 86: 119, 87: 90, 88: 113, 89: 126, 90: 106, 91: 105, 92: 105, 93: 97, 94: 99, 95: 108, 96: 107, 97: 100, 98: 96, 99: 108, 100: 98}


# Items generation

In [2]:
import random

def generate_item_data(num_items):
    volumes = {}
    demand = {}
    costs = {}
    alpha = {}
    beta = {}
    price = {}
    display_min = {}

    for i in range(1, num_items + 1):
        volumes[i] = int(random.uniform(0.01, 10))  # Random volume for item i
        demand[i] = int(random.randint(1, 16))  # Random demand for item i
        costs[i] = int(random.uniform(1.0, 101.0))  # Random cost for item i
        price[i] = 2 * costs[i]
        display_min[i] = int(random.uniform(50,70))

        for j in range(1, num_items + 1):
            if i == j:
                alpha[(j, i)] = 0.0
                beta[(j, i)] = 0.0
            else:
                alpha[(j, i)] = float("{:.2f}".format(random.uniform(0.1, 0.9)))  # Random alpha for item j -> item i
                beta[(j, i)] = float("{:.2f}".format(random.uniform(0.1, 0.9)))  # Random beta for item j -> item i

    return volumes, demand, costs, alpha, beta, price, display_min


# Specify the number of items
num_items = 20

# Generate item data
volumes, demand, costs, alpha, beta,price, display_min = generate_item_data(num_items)


# Resolution algorithms

In [8]:
import xpress as xp
import numpy as np
import time

def solution_stability(N_scenarios, num_items):
    # defining the problem
    volumes, demand, costs, alpha, beta, price, display_min = generate_item_data(num_items)
    KT_values = scenario_generation(N_scenarios)

    # Define the set of items
    N = list(range(1, num_items + 1))  # Example set of items
    lim_prod = 7
    # Define the set of scenarios
    Omega = list(range(1, len(KT_values) + 1))  # Example set of scenarios

    w = volumes
    d = demand
    alpha = alpha
    beta = beta
    p = price
    c = costs
    display_min = display_min
    C = 100 * np.dot(np.array(list(d.values())), np.array(list(w.values())))  # Example budget constraint
    M = 1000000000000000000000000  # Example capacity constraint
    delta = 0.7  # Example substitution constraint
    KT = KT_values

    # Define the model
    model = xp.problem(name="Stochastic Assortment Optimization")

    # Define the decision variables
    x = np.array([xp.var(vartype=xp.binary, name=f'x_{i}') for i in N])
    m = {(i, omega): xp.var(vartype=xp.integer, lb=0, name=f'm_{i}_{omega}') for omega in Omega for i in N}
    u = np.array([xp.var(vartype=xp.integer, lb=0, name=f'u_{i}') for i in N])
    mu = {(i, omega): xp.var(vartype=xp.binary, name=f'mu_{i}_{omega}') for omega in Omega for i in N}
    mu_1 = {(i, omega): xp.var(vartype=xp.binary, name=f'mu_1_{i}_{omega}') for omega in Omega for i in N}
    v_plus = {(i, omega): xp.var(vartype=xp.integer, lb=0, name=f'v_plus_{i}_{omega}') for omega in Omega for i in N}
    v_minus = {(i, omega): xp.var(vartype=xp.integer, lb=0, name=f'v_minus_{i}_{omega}') for i in N for omega in Omega}
    q_dict = {(i, j, omega): xp.var(vartype=xp.integer, lb=0, name=f'q_{i}_{j}_{omega}') for i in N for j in N for omega in Omega}
    s_dict = {(i, j, omega): xp.var(vartype=xp.integer, lb=0, name=f's_{i}_{j}_{omega}') for i in N for j in N for omega in Omega}

    model.addVariable(x)
    model.addVariable(m)
    model.addVariable(u)
    model.addVariable(v_plus)
    model.addVariable(v_minus)
    model.addVariable(q_dict)
    model.addVariable(s_dict)
    model.addVariable(mu)
    model.addVariable(mu_1)

    # Define the objective function
    expected_profit = xp.Sum(xp.Sum(m[(i, omega)] * p[i] for i in N) for omega in Omega) / len(Omega)
    assortment_substitution_profit = xp.Sum(
        xp.Sum(xp.Sum(p[i] * delta * alpha[(j, i)] * q_dict[(j, i, omega)] for i in N) for j in N) for omega in Omega
    ) / len(Omega)
    stockout_substitution_profit = xp.Sum(
        xp.Sum(xp.Sum(p[i] * delta * beta[(j, i)] * s_dict[(j, i, omega)] for i in N) for j in N) for omega in Omega
    ) / len(Omega)
    cost_of_storage = xp.Sum(c[i] * u[i - 1] for i in N)

    model.setObjective(
        expected_profit + stockout_substitution_profit + assortment_substitution_profit - cost_of_storage,
        sense=xp.maximize,
    )

    # Define the constraints
    budget_constraint = xp.Sum(w[i] * u[i - 1] for i in N) <= C
    display_constraint = [u[i - 1] >= display_min[i] * x[i - 1] for i in N]
    nbr_prods_constraint = xp.Sum(x[i - 1] for i in N) <= lim_prod

    capacity_constraints = [u[i - 1] <= M * x[i - 1] for i in N]  # Changed to a list comprehension
    nullity_constraints = [x[i - 1] <= u[i - 1] for i in N]

    min_constraints = []
    for i in N:
        for omega in Omega:
            min_constraints.append(m[(i, omega)] == xp.min(u[i - 1], KT[omega] * d[i]))

    assortment_constraints = []
    for omega in Omega:
        for i in N:
            for j in N:
                if j == i:
                    q_dict[j, i, omega] = 0
                if j != i:
                    assortment_constraints.append(q_dict[j, i, omega] <= M * x[i - 1])
                    assortment_constraints.append(q_dict[j, i, omega] <= M * (1 - x[j - 1]))

    lost_sales_constraints = []
    for omega in Omega:
        for i in N:
            lost_sales_constraints.append(
                v_plus[(i, omega)] - v_minus[(i, omega)] == m[(i, omega)] + xp.Sum(
                    q_dict[(j, i, omega)] for j in N if j != i
                ) - u[i - 1]
            )
            lost_sales_constraints.append(v_plus[(i, omega)] <= M * mu[(i, omega)])
            lost_sales_constraints.append(v_minus[(i, omega)] <= M * mu_1[(i, omega)])
            lost_sales_constraints.append(mu[(i, omega)] + mu_1[(i, omega)] == 1)

    stock_out_constraints = []
    for i in N:
        for j in N:
            for omega in Omega:
                if i == j:
                    s_dict[i, j, omega] = 0
                if j != i:
                    stock_out_constraints.append(s_dict[(j, i, omega)] <= M * x[i - 1])
                    stock_out_constraints.append(s_dict[(j, i, omega)] <= M * x[j - 1])

    global_constraints = []
    for omega in Omega:
        for i in N:
            global_constraints.append(
                xp.Sum(
                    m[(i, omega)] + xp.Sum(q_dict[(i, j, omega)] for j in N if j != i) + xp.Sum(
                        s_dict[(i, j, omega)] for j in N if j != i
                    )
                ) == KT[omega] * d[i]
            )

    global_capa_constraints = []
    for omega in Omega:
        for i in N:
            global_capa_constraints.append(
                xp.Sum(
                    m[(i, omega)] + xp.Sum(q_dict[(j, i, omega)] for j in N if j != i) + xp.Sum(
                        s_dict[(j, i, omega)] for j in N if j != i
                    )
                ) <= u[i - 1]
            )

    model.addConstraint(budget_constraint)
    model.addConstraint(capacity_constraints)
    model.addConstraint(nullity_constraints)
    model.addConstraint(assortment_constraints)
    model.addConstraint(min_constraints)
    model.addConstraint(lost_sales_constraints)
    model.addConstraint(stock_out_constraints)
    model.addConstraint(nbr_prods_constraint)
    model.addConstraint(display_constraint)
    model.addConstraint(global_constraints)
    model.addConstraint(global_capa_constraints)

    # Solve the problem
    start_time = time.time()
    model.solve()
    end_time = time.time()
    runtime = end_time - start_time

    # Get the optimal objective value
    optimal_objective_value = model.getObjVal()
    print("Optimal objective:", optimal_objective_value)

    # Get the optimal decision variable values
    optimal_x = {i: model.getSolution(x[i - 1]) for i in N}
    optimal_u = {i: model.getSolution(u[i - 1]) for i in N}
    optimal_s = {(i, j, omega): model.getSolution(s_dict[(i, j, omega)]) for i in N for j in N if j != i for omega in Omega}
    optimal_q = {(i, j, omega): model.getSolution(q_dict[(i, j, omega)]) for i in N for j in N if j != i for omega in Omega}
    optimal_m = {(i, omega): model.getSolution(m[(i, omega)]) for i in N for omega in Omega}
    optimal_v_plus = {(i, omega): model.getSolution(v_plus[(i, omega)]) for i in N for omega in Omega}
    optimal_v_minus = {(i, omega): model.getSolution(v_minus[(i, omega)]) for i in N for omega in Omega}
    optimal_mu = {(i, omega): model.getSolution(mu[(i, omega)]) for i in N for omega in Omega}
    optimal_mu_1 = {(i, omega): model.getSolution(mu_1[(i, omega)]) for i in N for omega in Omega}

    return (optimal_objective_value, optimal_x, optimal_u)


In [9]:
solutions = []

In [2]:
input_size = []
N_prod = 20
for i in range (1,501,10):
    input_size.append((i, N_prod))


[(1, 20),
 (11, 20),
 (21, 20),
 (31, 20),
 (41, 20),
 (51, 20),
 (61, 20),
 (71, 20),
 (81, 20),
 (91, 20),
 (101, 20),
 (111, 20),
 (121, 20),
 (131, 20),
 (141, 20),
 (151, 20),
 (161, 20),
 (171, 20),
 (181, 20),
 (191, 20),
 (201, 20),
 (211, 20),
 (221, 20),
 (231, 20),
 (241, 20),
 (251, 20),
 (261, 20),
 (271, 20),
 (281, 20),
 (291, 20),
 (301, 20),
 (311, 20),
 (321, 20),
 (331, 20),
 (341, 20),
 (351, 20),
 (361, 20),
 (371, 20),
 (381, 20),
 (391, 20),
 (401, 20),
 (411, 20),
 (421, 20),
 (431, 20),
 (441, 20),
 (451, 20),
 (461, 20),
 (471, 20),
 (481, 20),
 (491, 20)]

In [18]:
updatable_solutions = []
for inp in input_size:
    a = solution_stability(inp[0], inp[1])
    solutions.append(a)
    updatable_solutions.append(a)

Original problem size
   linear:    1722 rows, 941 columns, 5377 linear coefficients
   nonlinear: 20 coefficients, 120 tokens
Nonlinear presolve
   checking equivalence to general constraints ABS/MIN/MAX and PWL reformulations
   converted 20 MIN/MAX to optimizer MIP constructs
   converted 20 formulas to linear constraints
   problem is equivalent, reformulated as a MIP problem
Presolved problem size
   linear:    1722 rows, 961 columns, 5397 linear coefficients
Problem is nonlinear presolved
Maximizing problem using Xpress-Optimizer
FICO Xpress v9.0.0, Hyper, solve started 13:57:12, Jul 15, 2023
Heap usage: 1511KB (peak 1703KB, 1135KB system)
Maximizing MILP Stochastic Assortment Optimization using up to 8 threads and up to 63GB memory, with these control settings:
OUTPUTLOG = 1
IFCHECKCONVEXITY = 0
Original problem has:
      1722 rows          961 cols         5397 elements       940 entities
        20 gencons
Presolved problem has:
      1299 rows          877 cols         4168 